# Renewable.ninja API
from https://www.renewables.ninja/documentation/api/python-example

Limits in requests:
- Chronic max length: 1 year
- Max Request per minute: 6
- Max Request per hour: 50

Since this is generated from Merra weather database at one hour resolution, we will probably not have better resolution than 1 hour.
The available histopry goes back to the year 2000 until now



## Generate 1-year solar chronic at 1 hour resolution given coordinates

### Select solar coordinates

In [ ]:
#Path to change
#folium Draw API will save the file in your local Download repository and we will need to retrieve it
import os
import requests
import pandas as pd
import json

import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

YourDownloadPath=os.path.join(os.path.expanduser('~'), 'Downloads')
solarCoordinatesFileName='solarCoordinates.json'
CASE='case118_l2rpn_new'
year='2012'#the year on which to generate chronics
filePath=os.path.join(YourDownloadPath, solarCoordinatesFileName)

In [ ]:
token = '57a51353d6f26d8ad333fad459333bdd402cf19f' # Use your token by subscribing to renewable ninja API
yourOutputPath = os.path.join("/Users/antoinemarot/dev/ChroniX2Grid/chronix2grid/input/kpi/France/renewable_ninja",CASE)

try: 
    os.makedirs(yourOutputPath, exist_ok = True) 
    print("Directory '%s' created successfully" %directory) 
except OSError as error: 
    print("Directory '%s' can not be created") 



In [ ]:
# Read prods charac to associate chronics to usecase generators
prods_charac = pd.read_csv(os.path.join('../../../../generation/input',CASE,'prods_charac.csv'), sep = ',')
prods_charac.head()

In [ ]:
# Select only solar generators
solar_gen = prods_charac[prods_charac['type']=='solar'].sort_values(by = 'zone')
solar_gen

### Chose locations on map
You should try to simulate the locations in prods_charac (3 regions). Set points in the same order as they appear in prods_charac above

In [ ]:
import folium
from folium.plugins import Draw, MousePosition, MiniMap, LocateControl


m = folium.Map()


if os.path.exists(filePath):
    os.remove(filePath)

draw = Draw(export=True, filename=solarCoordinatesFileName)

draw.add_to(m)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m)

minimap = MiniMap()
m.add_child(minimap)

LocateControl(auto_start=True).add_to(m)

m


#### Export your locations with the export button

#### Extract your locations

In [ ]:
#read JSON
import geopandas as gpd

gdfSolar = gpd.read_file(filePath)
gdfSolar['longitude']=gdfSolar.geometry.x
gdfSolar['latitude']=gdfSolar.geometry.y


# Add usecase generators
for col in ['name','zone']:
    gdfSolar[col] = solar_gen[col].values

gdfSolar.head()

### Get your solar chronic from renewable ninja per location and export it per year
So that it is directly interpretable by KPIs

In [ ]:
import requests
import pandas as pd
import json


api_base = 'https://www.renewables.ninja/api/'

s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + token}

##
# PV example
##

url = api_base + 'data/pv'
dfSolarLocation=pd.DataFrame()

In [ ]:

import time


sleep_time = 62


s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + token}

dfSolarLocation=pd.DataFrame()


for i, markerIndex in enumerate(range(len(gdfSolar))):
    
    if i % 5 == 0:
        time.sleep(sleep_time)
    
    ##
    # PV example
    ##

    #year=SixYears[0]
    latitude=gdfSolar['latitude'][markerIndex]
    longitude=gdfSolar['longitude'][markerIndex]
    name = gdfSolar['name'][markerIndex]
    
    print(name, latitude, longitude)
    
    date_from=year+'-01-01'
    date_to=year+'-12-31'

    args = {
        'lat': latitude,
        'lon': longitude,
        'date_from': date_from,
        'date_to': date_to,
        'dataset': 'merra2',
        'capacity': 1.0,
        'system_loss': 0.1,
        'tracking': 0,
        'tilt': 35,
        'azim': 180,
        'format': 'json'
    }

    r = s.get(url, params=args)
    print(r)

    # Parse JSON to get a pandas.DataFrame of data and dict of metadata
    parsed_response = r.json()

    dataSolarChronic = pd.read_json(json.dumps(parsed_response['data']), orient='index')
    dataSolarChronic.index=pd.to_datetime(dataSolarChronic.index,utc=True) #removing the year to get same index for all years

    dfSolarLocation[name]=dataSolarChronic['electricity']
    
dfSolarLocation.to_csv(os.path.join(yourOutputPath,'solar_'+str(year)+'.csv'), sep = ';', index = False)

In [ ]:
dfSolarLocation.iplot()

## Generate 1-year wind chronic at 1 hour resolution given ccordinates

### Select wind coordinates

In [ ]:
windCoordinatesFileName='windCoordinates.json'
filePath=os.path.join(YourDownloadPath, windCoordinatesFileName)

In [ ]:
# Select only wind generators
wind_gen = prods_charac[prods_charac['type']=='wind'].sort_values(by = 'zone')
wind_gen

In [ ]:
import folium
from folium.plugins import Draw, MousePosition, MiniMap, LocateControl

m = folium.Map()


if os.path.exists(filePath):
    os.remove(filePath)

draw = Draw(export=True, filename=windCoordinatesFileName)

draw.add_to(m)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m)

minimap = MiniMap()
m.add_child(minimap)

LocateControl(auto_start=True).add_to(m)

m


#### Export your locations with the export button

#### Extract your locations

In [ ]:
#read JSON
import geopandas as gpd
import pandas as pd

gdfWind = gpd.read_file(filePath)
gdfWind['longitude']=gdfWind.geometry.x
gdfWind['latitude']=gdfWind.geometry.y

#O ajoute les colonnes
for col in ['name','zone']:
    gdfWind[col] = wind_gen[col].values

gdfWind.head(10)

### Get wind solar chronic from renewable ninja


#### Select your location in the gdf dataframe


#### Get Chronics from api

In [ ]:
##
# Wind example
##
api_base = 'https://www.renewables.ninja/api/'
url = api_base + 'data/wind'
s = requests.session()
# Send token header with each request
token = '57a51353d6f26d8ad333fad459333bdd402cf19f'
s.headers = {'Authorization': 'Token ' + token}
args = {
    'lat': latitude,
    'lon': longitude,
    'date_from': '2015-01-01',
    'date_to': '2015-12-31',
    'capacity': 1.0,
    'height': 100,
    'turbine': 'Vestas V80 2000',
    'format': 'json'
}

r = s.get(url, params=args)

parsed_response = json.loads(r.text)
dataWind = pd.read_json(json.dumps(parsed_response['data']), orient='index')
metadataWind = parsed_response['metadata']

In [ ]:

import time

api_base = 'https://www.renewables.ninja/api/'
url = api_base + 'data/wind'
sleep_time = 62


s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + token}

dfWindLocation=pd.DataFrame()


for i, markerIndex in enumerate(range(len(gdfWind))):
    
    if i % 5 == 0:
        time.sleep(sleep_time)
    
    ##
    # PV example
    ##

    #year=SixYears[0]
    latitude=gdfWind['latitude'][markerIndex]
    longitude=gdfWind['longitude'][markerIndex]
    name = gdfWind['name'][markerIndex]
    
    print(name, latitude, longitude)
    
    date_from=year+'-01-01'
    date_to=year+'-12-31'

    args = {
        'lat': latitude,
        'lon': longitude,
        'date_from': date_from,
        'date_to': date_to,
        'capacity': 1.0,
        'height': 100,
        'turbine': 'Vestas V80 2000',
        'format': 'json'
    }

    r = s.get(url, params=args)
    print(r)

    # Parse JSON to get a pandas.DataFrame of data and dict of metadata
    parsed_response = r.json()

    dataWindChronic = pd.read_json(json.dumps(parsed_response['data']), orient='index')
    dataWindChronic.index=pd.to_datetime(dataWindChronic.index,utc=True) #removing the year to get same index for all years

    dfWindLocation[name]=dataWindChronic['electricity']

dfWindLocation.to_csv(os.path.join(yourOutputPath,'wind_'+str(year)+'.csv'), sep = ';', index = False)

In [ ]:
dfWindLocation.iplot()